In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
sys.path.append('./../../designer/script/')
sys.path.append('./../')
sys.path.append('./../../')
from film import TwoMaterialFilm, FreeFormFilm
from spectrum import Spectrum
from design import BaseDesign, DesignForSpecSimple
from optimizer.adam import AdamThicknessOptimizer, AdamFreeFormOptimizer
from utils.loss import calculate_RMS_f_spec
import os
import torch

from analyze_utils.structure import plot_layer_thickness
import dill
import tqdm



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
def make_antireflector_design():
    inc_angs = np.linspace(40, 80, 2)
    # wls = np.linspace(700, 800, 500) # when wls = 50, ~100 min
    wls = np.linspace(400, 1200, 500) # DBR
    R = np.zeros(wls.shape[0] , dtype='float')

    target_spec = [Spectrum(inc_ang, wls, R) for inc_ang in inc_angs]
    return target_spec

target = make_antireflector_design()

np.random.seed(0)
d_init = np.random.rand(50) * 100
film = TwoMaterialFilm(1.05, 'TiO2', 'Si', d_init)

adam = AdamThicknessOptimizer(
    film, 
    target, 
    max_steps=100, 
    remove_nonpos_during_optm=True, 
    show=True, 
    show_condition=lambda i: i % 10 == 0, 
    alpha=1, 
    # record=lambda i: i == 1999
)

adam.optimize()

  1%|          | 1/100 [00:00<00:37,  2.65it/s]

iter 0, loss 0.6148747689654561
Layer removed.


 12%|█▏        | 12/100 [00:02<00:12,  6.99it/s]

iter 10, loss 0.57177675842079


 22%|██▏       | 22/100 [00:03<00:07, 10.50it/s]

...

iter 80, loss 0.2579094425153436


 92%|█████████▏| 92/100 [00:14<00:00,  9.15it/s]

iter 90, loss 0.22628486311560514


100%|██████████| 100/100 [00:15<00:00,  6.43it/s]


[(<film.TwoMaterialFilm at 0x7fe10e925b40>,), (0.6202290810408689,)]

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(2, 1, gridspec_kw={'height_ratios': (5, 1), 'hspace': 0.5})

film_specs = [film.get_spec(inc_ang=inc_ang, wls=np.linspace(400, 1200, 500)) for inc_ang in np.linspace(40., 80., 100)]
r_spec = np.array([film_spec.get_R() for film_spec in film_specs])

ax = axes[0]
im = ax.imshow(
    r_spec, 
    cmap='coolwarm', 
    aspect='auto', 
    extent=[400., 1200., 80., 40.], 
    vmax=0.2, 
    vmin=0
)
ax.invert_yaxis()

# colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical', label='R')

ax.set_title(f'MSE: {calculate_RMS_f_spec(film, target): .5f}, final layer number: {film.get_layer_number()}')

# print(film.get_layer_number())
# film.remove_negative_thickness_layer(zero=10)
# print(film.get_layer_number())
# ax.plot(film.get_spec().WLS, film.get_spec().get_R(), label='After removal')

ax.set_xlabel('Wavelength / nm')
ax.set_ylabel('Incident angle / deg')
fig.set_size_inches(5, 3)

plot_layer_thickness(film, ax=axes[1])
axes[1].set_ylabel('n')
axes[1].set_xlabel('Thickness / nm')

fig.set_size_inches(5, 6)